# 第4回講義 宿題

## 課題. MNISTデータセットを多層パーセプトロン(MLP)で学習せよ

### 注意
- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください (必要なものは全てhomework関数に入れてください)
- 解答提出時には Answer Cell の内容のみを提出してください

- MLPの実装にTensorflowなどのライブラリを使わないでください

### ヒント
- 出力yはone-of-k表現
- 最終層の活性化関数はソフトマックス関数, 誤差関数は多クラス交差エントロピー
- 最終層のデルタは教科書参照

次のセルのhomework関数を完成させて提出してください

# Answer Cell

In [ ]:
def homework(train_X, train_y, test_X):
   
    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def deriv_sigmoid(x):
        return sigmoid(x)*(1 - sigmoid(x))
    
    def softmax(x):
        exp_x = np.exp(x)
        return exp_x/np.sum(exp_x, axis=1, keepdims=True)

    def deriv_softmax(x):
        return softmax(x)*(1 - softmax(x))
    
    def oneHotEnding(train_y):
        y_size = len(train_y)
        y = np.zeros((y_size, 10))
        y[np.arange(y_size), train_y] = 1
        return y
    
    def train(x, t, eps=0.2):
        nonlocal W1, b1, W2, b2
        
        # Forward Propagation Layer1
        u1 = np.matmul(x, W1) + b1
        z1 = sigmoid(u1)
    
        # Forward Propagation Layer2
        u2 = np.matmul(z1, W2) + b2
        z2 = softmax(u2)
    
        # Back Propagation (Cost Function: Negative Loglikelihood)
        y = z2
        cost = np.sum(-t*np.log(y))
        delta_2 = y - t
        delta_1 = deriv_sigmoid(u1) * np.matmul(delta_2, W2.T)

        # Update Parameters Layer1
        dW1 = np.matmul(x.T, delta_1)
        db1 = np.matmul(np.ones(len(x)), delta_1)
        W1 = W1 - eps*dW1
        b1 = b1 - eps*db1
    
        # Update Parameters Layer2
        dW2 = np.matmul(z1.T, delta_2)
        db2 = np.matmul(np.ones(len(z1)), delta_2)
        W2 = W2 - eps*dW2
        b2 = b2 - eps*db2

        return cost
    
    def test(x):
        nonlocal W1, b1, W2, b2
        # Forward Propagation Layer1
        u1 = np.matmul(x, W1) + b1
        z1 = sigmoid(u1)
    
        # Forward Propagation Layer2
        u2 = np.matmul(z1, W2) + b2
        z2 = softmax(u2)
    
        y = z2
        return y
    

    # initialize
    # OneHotEncoding
    train_y = oneHotEnding(train_y)
    # normalize train array
    norm = np.linalg.norm(train_X, ord=2, axis=1)
    train_X = train_X / norm[:, np.newaxis]
    # normalize test array
    norm = np.linalg.norm(test_X, ord=2, axis=1)
    test_X = test_X / norm[:, np.newaxis]
    # Layer1 weights
    W1 = np.random.uniform(low=-0.08, high=0.08, size=(784, 32)).astype('float32')
    b1 = np.zeros(32).astype('float32')
    # Layer2 weights
    W2 = np.random.uniform(low=-0.08, high=0.08, size=(32, 10)).astype('float32')
    b2 = np.zeros(10).astype('float32')
    
    # Epoch
    for epoch in range(42):
        for x, y in zip(train_X, train_y):
            cost = train(x[np.newaxis, :], y[np.newaxis, :])
            # print(cost)
    
    pred_y = test(test_X)
    # unOneHotEncoding
    pred_y = [np.argmax(y) for y in pred_y]
    
    return pred_y

- 以下のvalidate_homework関数を用いてエラーが起きないか動作確認をして下さい。
- 提出に際して、score_homework関数で60分で実行が終わることを確認して下さい。
- 評価は以下のscore_homework関数で行われますが、random_stateの値は変更されます。

# Checker Cell (for student)

In [ ]:
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

import numpy as np

def load_mnist():
    mnist = fetch_mldata('MNIST original')
    mnist_X, mnist_y = shuffle(mnist.data.astype('float32'),
                               mnist.target.astype('int32'), random_state=42)

    mnist_X = mnist_X / 255.0

    return train_test_split(mnist_X, mnist_y,
                test_size=0.2,
                random_state=42)

def validate_homework():
    train_X, test_X, train_y, test_y = load_mnist()

    # validate for small dataset
    train_X_mini = train_X[:100]
    train_y_mini = train_y[:100]
    test_X_mini = test_X[:100]
    test_y_mini = test_y[:100]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    print(f1_score(test_y_mini, pred_y, average='macro'))

def score_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(test_y, pred_y, average='macro'))

In [ ]:
validate_homework()
# score_homework()